In [1]:
#import libraries

from bs4 import BeautifulSoup
import selenium.webdriver
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import datetime
import requests
import time
import smtplib

In [2]:
#Connect to the website and pull the data

URL = 'https://www.amazon.com/s?k=headphones'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

page_soup = BeautifulSoup(page.content,"html.parser")

page_soup_prettify = BeautifulSoup(page_soup.prettify(), "html.parser")

In [5]:
links = page_soup_prettify.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal', href=True)

data = []
df = pd.DataFrame(data)

for link in links:
    # Get the URL from the href attribute
    item_url = "https://www.amazon.com" + link['href']
    print(item_url)
        
    # Do something with the URL (print, store, etc.)
    URL = item_url

    page = requests.get(URL, headers=headers)

    try:
        if page.status_code == 200:

            page_soup = BeautifulSoup(page.content,"html.parser")

            page_soup_prettify = BeautifulSoup(page_soup.prettify(), "html.parser")

            product_title = page_soup_prettify.find(id='productTitle').get_text()
            product_title = product_title.strip()
            product_price_1 = page_soup_prettify.find(class_='a-price-whole')

            if product_price_1 :
                # Find and extract text from all direct child elements that are not spans
                product_price_1= ' '.join([child.get_text(strip=True) for child in product_price_1.children if child.name != 'span'])
            else:
                print("No element with the specified parent class found.")


            product_price_2 = page_soup_prettify.find(class_='a-price-fraction').get_text().strip()
            product_price_1 = product_price_1.strip()

            # Concatenate the two parts and remove extra spaces
            whole_price = product_price_1 + "." + product_price_2
            data.append({'Item URL': item_url, 'Product Title': product_title, 'Product Price': whole_price})
            time(10)
        else:
            print(f"Error: Unable to fetch the page. Status code: {page.status_code}")

    except page.ConnectionError as e:
        print(f"ConnectionError for URL: {URL}. Skipping to the next one.")
        continue


# Save the DataFrame to an Excel file
df.to_excel('amazon_data.xlsx', index=False)

print("Data saved to 'amazon_data.xlsx'")
    

ModuleNotFoundError: No module named 'openpyxl'